# Django ORM - optymalizacja zapytań

Najpierw linijki, które pozwolą nam swobodnie korzystać z Django w notatniku Jupyter.

In [ ]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'intro.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

### 1. Złączenia (join)

Nawiązując do poprzedniego notatnika (ORM IV), w jaki sposób, za pomocą dotychczas poznanych narzędzi moglibyś zrealizować joina?

Weźmy tabelki Framework i language

In [ ]:
from orm_app.models import Framework, Language

Z tabelki Framework wyciągnijmy Django

In [ ]:
django_qs = Framework.objects.filter(name='django')

# na bazie wykona się taka sqlka
print(django_qs.query)

In [ ]:
django = django_qs[0]
django

In [ ]:
# jeżeli teraz chcemy wyciągnąć powiązany z wpisem django wpis z tabelki Language
python = django.language

# to django orm musi wykonać kolejne zapytanie na bazę
python

Załóżmy teraz, że chcemy wyświetlić język wszystkich frameworków zapisanych w bazie. Wykorzystując dotychczas poznane narzędzie kod mógłby wyglądać mniej więcej tak:

In [ ]:
frameworks = Framework.objects.all()

for framework in frameworks:
    print(framework.language)

Przy założeniu, że w bazie mamy zapisane 100 frameworków django będzie musiał uderzyć na bazę 101 razy (raz, żeby wyciągnąć wszystkie frameworki i później po razie na każdy framework, żeby wyciągnąć jego język). 

Znając klauzulę JOIN wiemy, że z jej użyciem możnabyłoby wszystkie potrzebne informacje wyciągnąc za pomocą jednego zapytania na bazę. Ma to szczególne znaczenie, kiedy wyciągamy dużo wpisów (np. 100 albo 1000). 1000 dodatkowych zapytań do bazy wygeneruje zauważalne wydłużenie czasu wykonywania kodu. Jak zmusić django orm do wykonania JOINa?

Za pomocą django orm **instrukcje join** możemy zrealizować na dwa sposoby:
- `select_related`
- `prefetch_related`

To, której metody będziemy chcieli użyć zależy od charaketru relacji wzdłuż, której złączamy tabelki.

#### select_related

`select_related` wykonuje trdycyjnego joina, bez żadnych dodatkowych usprawnień. Używamy w przypadku relacji o2o oraz o2m. Proba użycia na relacji m2m zakończy się błędem z przyczyn, o których poniżej.

In [ ]:
frameworks = Framework.objects.select_related()

# tym razem na bazie wykona się taka sqlka
print(frameworks.query)

Widzimy joina. Django orm wyciągnie z bazy informacje o wpisach powiązanych i powkłada te informacje do odpowiednich obiektów w taki sposób, że odwołując się do języka frameworka, orm nie będzie potrzebował, ponownie uderzyć na bazę.

In [ ]:
# tym razem ten kod wykona się bez uderzania na bazę
for framework in frameworks:
    print(framework.language)

Jeżeli chcemy zrobić joina tylko z wybranymi powiązaniami (a nie ze wszystkimi), dopisujemy nazwy wybranych kluczy obcych jako parametry pozycyjne metody select_related.

In [ ]:
frameworks = Framework.objects.select_related('language')
print(qs.query)  
# w tym przykładzie nie widzimy różnicy pomiędzy tym zapytaniem, a poprzednim ponieważ tabelka framework ma tylko 
# jedno powiązanie (z tabelką Language)

Jeżeli chcielibyśmy robić kolejnego joina z wykorzystaniem danych z tabeli zjoinowanej, możemy posłużyć się `field lookupami`. Założmy hipotetycznie, że tabelka language ma jeszcze kolumnę type, która jest kluczem obcym do tabelki Type (niech tabelka Type reprezentuje typ języka programowania, np. statyczny, dynamiczny). Jeżeli chcielibyśmy zjoinować również tą kolejną tabelkę, możemy to zrobić w sposób:

In [ ]:
frameworks = Framework.objects.select_related('language__type')
print(frameworks.query)
# W ten sposób możemy łańcuchować lookupy joinując dalsze tabelki

W wyniku wykonywania joinów czasami (zwłaszcza w przypadku relacji m2m) możemy wyciągać z bazy dużą ilość danych. Z myślą o takich przypadkach django orm udostępnia drugą metodę na zrealizowanie joina - `prefetch_related`.

#### prefetch_related

In [ ]:
frameworks = Framework.objects.prefetch_related()

print(frameworks.query)

Tutaj nie widzimy, żeby django zrobił jakiegoś joina, ale pod spodem django wykonał dwa zapytania na bazę. Jedno to to, którego sql widzimy powyżej. Drugie zapytanie było o wpisy powiązane. A następnie połączył te informacje ze sobą już po stronie pythona. W efekcie zamiast 101 zapytań zrobiliśmy dwa. 

`prefech_related` używamy kiedy nie chcemy uderzać wiele razy na bazę, a jednocześnie wiemy że w wyniku pojdynczego zapytania dostaniemy ogromną liczbę danych, których przetworzenie może zająć więcej czasu niż przetworzenie tej samej ilości danych, ale w dwóch (kilku) mniejszych zbiorach.

Jednym z najlepszych sposobów na śledzenie jakości zapytań bazodanowych po stronie Django jest użycie Django Debug Toolbar (https://django-debug-toolbar.readthedocs.io/en/latest/installation.html), dodatku do django, który monitoruje wiele parametrów widoku, wśród których znajduje się liczba zapytań sql i czas ich wykonania.

### 2. Ograniczanie wyniku - wybór wskazanych pól

Pracować będziemy na modelu postaci:

<code>
class Product(models.Model):
    title = models.CharField(max_length=100)
    manufacturer = models.CharField(max_length=100)
    price = models.IntegerField()
</code>
<code>
    a = models.CharField(max_length=100)
    b = models.CharField(max_length=100)
    c = models.CharField(max_length=100)
</code>
<code>    
    product_secret_id = models.CharField(max_length=100)
</code>

Dopisz powyższy model do jednej ze swoich aplikacji.

Zaimportujmy model

In [ ]:
from orm_app.models import Product

Dodajmy kilka wpisów

In [ ]:
product1 = Product.objects.create(
    title='test1',
    manufacturer='test',
    price=100,
    a='ala',
    b='ma',
    c='kota',
    product_secret_id='1111'
)

product2 = Product.objects.create(
    title='test2',
    manufacturer='test',
    price=10,
    a='kot',
    b='ma',
    c='ale',
    product_secret_id='2222'
)

product3 = Product.objects.create(
    title='test3',
    manufacturer='test',
    price=1,
    a='ewa',
    b='ma',
    c='psa',
    product_secret_id='3333'
)

I napiszmy zapytanie o wszystkie wpisy w tabelce.

In [ ]:
products = Product.objects.all()
print(products.query)

Powyższe zapytanie zwraca nam wartości WSZYSTKICH KOLUMN ze wszystkich wpisów w tabelce. Jak napisać zapytanie, które wyciągnie wartości tylko z wybranych kolumn?

Możemy oczywiście wyciągnąć z queryseta tylko te wartości, które nas interesują

In [ ]:
[(product.title, product.manufacturer, product.price) for product in products]

Ale w ten sposób wciąż wyciągamy z bazy znacznie więcej informacji niż jest nam potrzebne (i dopiero w pythonie wybieramy z tego wyciąganiętego zbiory tylko te informacje, które nas interesują). Może to generować szereg istotnych problemów:
- im więcej danych chcemy wyciągnąć z bazy, tym dłużej użytkownicy będą czakać na wykonanie zapytania (oraz przesyłanie wyciągniętych danych) 
- w czasie kiedy wykonywane jest zapytanie na bazie inne zapytania muszą czekać, co ogranicza liczbę użytkowników jaką nasza aplikacja może płynnie obsłużyc
- obecnie, często w rozwiązaniach chmurowych za wykonanie zapytania płaci się proporcjonalnie do czasu wykonywania tego zapytania na bazie, co jeszcze bardziej podnosi rangę problemu

Oczywistym staje się potrzeba zminimalizowania ilość danych przesyłanych pomiędzy bazą a aplikacją. Chcemy wyciągnąć z bazy wyłącznie te informacje, które są nam potrzebne (co zawsze jest dobrą praktyką).

Mamy 4 metody, których możemy w tym celu użyć:
- `only`
- `defer`
- `values`
- `values_list`

### only

In [ ]:
products = Product.objects.all().only('title', 'manufacturer', 'price')
print(products.query)

In [ ]:
print(products)

W wyniku otrzymujemy tradycyjnego queryseta, którego wartości to obiekty klasy `Product`.

Użycie only nie zabezpiecza nas jednak przed odpytywaniem bazy o pola, których nie wyciągneliśmy z bazy. W przypadku, gdybyśmy odwołali się do pola a wpisu, django orm po prostu drugi raz uderzy na baze i pobierze tę wartość.

In [ ]:
products[0].a

W skrajnych przypadkach może to prowadzić do wykonania setek/tysięcy dodatkowych zapytań na bazę. Dlatego należy zwrócić szczególną uwagę na to, żeby w metodzie only umieścić wszystkie pola modelu, które zamierzamy użyć.

Istnieje druga bardzo podobna metoda - `defer`.

### defer

 Metoda `defer` działa identycznie jak metoda `only` z dokładnością do tego, że jako parametr podajemy te wartości, których z bazy nie chcemy pobierać.

In [ ]:
products = Product.objects.all().defer('title', 'manufacturer', 'price')
print(products.query)

Widzimy, że w zapytaniu nie ma żadnego ze wskazanych pól (`title`, `manufacturer`, `price`). Ale wciąż możemy o nie odpytać. Będzie to jednak wymagało kolejnego uderzenia na bazę.

In [ ]:
products[0].title

Do dyspozycji mamy również drugą metodą - `values`, która w niektórych scenariuszach moze okazać się lepszym rozwiązaniem.

### values

In [ ]:
products = Product.objects.all().values('title', 'manufacturer', 'price')
print(products.query)

In [ ]:
print(products)

Tym razem dostaliśmy queryset, ale słowników (a nie obiektów klasy Product). 

In [ ]:
print(products[0])

In [ ]:
print(type(products[0]))

Korzyści:

1. słowniki pythonowe zajmują mniej pamięci niż obiekty modelu
2. nie jesteśmy już w stanie z pojedynczego elementu tego queryseta wyciągnąć wartość pola, którego nie zawarliśmy w parametrach metody `values`. 

In [ ]:
print(products[0]['title'])

Zamiast słowników możemy otrzymać to samo w postaci krotek za pomocą metody `values_list`.

### values_list

In [ ]:
products = Product.objects.all().values_list('title', 'manufacturer', 'price')
print(products.query)

In [ ]:
print(products)

In [ ]:
print(products[0])

In [ ]:
print(type(products[0]))

In [ ]:
W przypadku wyciągania tylko jednej wartości

In [ ]:
products = Product.objects.all().values_list('title')
products

Wynik możemy spłaszczyć za pomocą parametru `flat`

In [ ]:
products = Product.objects.all().values_list('title', flat=True)
products

### 4. Optymazliacja zapytań za pomocą wyrażeń F 

**F expressions** pozwalają nam na wykonywanie operacji na wybranym wpisie z poziomu bazy danych, bez konieczności wcześniejszego wyjmowania tego wpisu z bazy.

O co chodzi?

Zastanwómy się jak wygląda tradycyjny update na bazie.

In [ ]:
customer = Customer.objects.get(id=1)
print(f"Age before: {customer.age}")
customer.age += 1
customer.save()

In [ ]:
customer = Customer.objects.get(id=1)
print(f"Age after: {customer.age}")

In [ ]:
from django.db import connection

connection.queries[-2:]

Zwróć uwagę, że w celu zrobienia updatu musieliśmy wyciągnąć najpierw wpis (wpisy) z bazy, zaczytać go (je) do pamięci ram w pythonie, zrobić update a następnie zapisać do bazy. Nie wydaje się to być optymalnym rozwiązaniem. Po co to całe wczytywanie do pythona? Nie lepiej byłoby zrobić całość z poziomu bazy danych?

Tylko jak?

Za pomocą wyrażenia F.

In [ ]:
from django.db.models import F

F('name')+1

#### Wyrażenia F w metodzie update

In [ ]:
customer = Customer.objects.get(id=1)
customer.age = F('name')+1
customer.save()

In [ ]:
connection.queries[-1]

Nie ma to oczywiście większego sensu w przypadku updatu pojedynczego wpisu, ale gdyby update szedł po setkach wpisów to różnicę odczujemy wyraźnie.

Jak wyglądałoby zapytanie, gdybyśmy chcieli wykonać ten update na wszysktich większej liczbie wpisów ?

In [ ]:
Customer.objects.update(age=F('age')+1)

In [ ]:
connection.queries[-1]

Inne operatory matematyczne również zadziałają.

In [ ]:
Customer.objects.update(age=F('age')**2 - F('age')/2)

In [ ]:
connection.queries[-1]

Za pomocą wyrażeń F możemy odwoływać się do dowolnych pól z naszego modelu.

#### Wyrażenia F w metodzie filter

Wyrażenia F możemy też wykorzystywać w lookupach.

Przyjmijmy hipotetycznie, że model Customer posiada kolumnę `expected_age`, a my chcemy wyświetlić tylko tych `customer`, których wartość w kolumnie `age` jest większy od wartości w kolumnie `expected_age`.

<code>Customer.objects.filter(age__gt=???)</code>

W tym miejscu również świetnie sprawdzają się wyrażenia F.

<code>Customer.objects.filter(age__gt=F('expected_age'))</code>

Bez wyrażeń F trzeba byłoby wczytać wszystkie wartości do pythona, przeiterować się po nich i porównać w pythonie, a następnie na podstawie porównania porobić updaty do bazy. W zależności od liczby wpisów takie rozwiązanie może być nawet o kilka rzędów wielkości wolniejsze.